### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from configuration import Config
from statistics import Statistics

### Setup the environment

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Configure a genetic algorithm

In [4]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### Create a sample population

In [8]:
sample_pop = configs.params['enc_type'].initialize()
print(sample_pop[0])

{'gene': array([77, 65,  1, 73, 44, 20, 46, 30, 26, 59, 82, 19, 50, 76, 37,  2, 55,
        4, 13, 84, 36, 60, 40,  8, 11, 25, 57, 78, 63,  0, 39, 75, 70, 24,
       68,  7, 33,  3, 38, 74, 17, 42,  5, 81, 87, 54, 32,  6, 91, 31, 58,
       18, 92, 80, 88, 41, 86, 51, 21, 15, 28, 48, 90, 94, 79, 22, 61, 62,
       34, 66, 69, 52, 10, 83, 96, 97, 47,  9, 29, 14, 64, 49, 72, 67, 99,
       43, 27, 89, 71, 16, 95, 45, 12, 53, 23, 93, 35, 98, 56, 85]), 'fitness': array([0]), 'meta': {}}


### Define a custom evaluation function

Your evaluation function should update all fitness values of an individual. Sample evaluation functions for the predefined problems can be found in ```evaluate.py```



In [6]:
def custom_eval(self, pop):
    return pop

configs.params['eval_type'] = cust_eval

In [6]:
sample_pop = configs.params['eval_type'](configs, sample_pop)
print(sample_pop[0])

{'gene': array([7, 3, 6, 1, 5, 0, 4, 2]), 'fitness': array([4]), 'meta': {}}


### Define a custom visualization

Your visualization function should define a phenotypic representation of an individual using the ```dash_core_components``` or ```dash_bootstrap_components``` libraries. Sample visualization functions for the predefined problems can be found in ```visualize.py```


In [7]:
def cust_vis(ind):
    return go.Figure()
configs.params['cust_vis'] = cust_vis

### Run the genetic algorithm

In [5]:
stats = Statistics(configs.params)
main_app = app_file.create_app(configs, stats)
main_app.run_server(mode='inline')

### Analyze the resulting population

In [ ]:
population = statistics.posthoc['population']